# Data Extraction and Filtering

### Loading libraries and data

In [38]:
#Importing libraries for SQL
import sqlite3

#Libraries for basic data prep and eda
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)
pd.set_option('display.width', 160)
pd.set_option('display.max_colwidth', 30)

In [26]:
#Creating a database file
conn = sqlite3.connect("players_data.db")

#Creating a cursor to execute SQL commands
cursor = conn.cursor()

In [27]:
#Loading CSV file
df = pd.read_csv("/Users/dan/Documents/Personal Projects/players_data-2024_2025.csv")

#Storing the data in an SQLite table
df.to_sql("players", conn, if_exists="replace", index=False)

2854

In [28]:
#Checking table
query = "SELECT * FROM players LIMIT 5;"

df_sql_preview = pd.read_sql(query, conn)
print(df_sql_preview)

   Rk             Player   Nation    Pos        Squad                Comp   Age    Born  MP  Starts  Min  90s  Gls  Ast  G+A  G-PK  PK  PKatt  CrdY  CrdR  \
0   1         Max Aarons  eng ENG     DF  Bournemouth  eng Premier League  25.0  2000.0   3       1   86  1.0    0    0    0     0   0      0     0     0   
1   2         Max Aarons  eng ENG  DF,MF     Valencia          es La Liga  25.0  2000.0   4       1  120  1.3    0    0    0     0   0      0     2     0   
2   3     Rodrigo Abajas   es ESP     DF     Valencia          es La Liga  22.0  2003.0   1       1   65  0.7    0    0    0     0   0      0     1     0   
3   4     James Abankwah   ie IRL  DF,MF      Udinese          it Serie A  21.0  2004.0   6       0   88  1.0    0    0    0     0   0      0     1     0   
4   5  Keyliane Abdallah   fr FRA     FW    Marseille          fr Ligue 1  19.0  2006.0   1       0    3  0.0    0    0    0     0   0      0     0     0   

    xG  npxG  xAG  npxG+xAG  PrgC  PrgP  PrgR  G+A-PK  xG

### Filtering and aggregation

In [29]:
#Filtering data and column selection using SQLite

query = """
SELECT
    Player name,
    Pos position,
    Nation nation,
    Age age,
    Squad club,
    Comp league,
    "Min" mins_played,
    "90s" ninety_mins_played,
    Gls goals_scored,
    PK penalty_goals,
    npxG non_pen_exp_goals,
    xG exp_goals,
    Sh total_shots,
    SoT shots_on_target,
    "G/Sh" goals_per_shot,
    "G/SoT" goals_per_sot,
    "SoT%" shot_on_target_perc,
    Dist avg_shot_distance,
    Ast assists,
    xAG exp_assists,
    KP key_passes,
    "SCA90" shot_creation_per_90,
    "GCA90" goal_creation_per_90,
    "Cmp%" pass_completion_perc,
    Touches total_touches,
    PrgC progressive_carries,
    Carries total_carries,
    Dis times_dispossessed,
    Rec passes_received,
    "Att 3rd" touches_in_attacking_third,
    "Att Pen" touches_in_pen,
    PrgR progressive_runs,
    Won aerial_duels_won,
    Lost aerial_duels_lost,
    "Won%" aerial_duel_win_perc,
    Tkl tackles,
    "Tkl+Int" tackles_and_interceptions
FROM players
WHERE Pos LIKE '%FW%'
"""

filtered_df = pd.read_sql(query, conn)
print(filtered_df.head())
conn.close()

                name position   nation   age       club      league  mins_played  ninety_mins_played  goals_scored  penalty_goals  non_pen_exp_goals  \
0  Keyliane Abdallah       FW   fr FRA  19.0  Marseille  fr Ligue 1            3                 0.0             0              0                0.0   
1      Himad Abdelli    MF,FW   dz ALG  25.0     Angers  fr Ligue 1         2842                31.6             6              3                3.2   
2     Matthis Abline       FW   fr FRA  22.0     Nantes  fr Ligue 1         2768                30.8             9              1                7.7   
3  Zakaria Aboukhlal    MF,FW   ma MAR  25.0   Toulouse  fr Ligue 1         1901                21.1             7              1                8.6   
4      Tammy Abraham       FW  eng ENG  27.0       Roma  it Serie A            1                 0.0             0              0                0.0   

   exp_goals  total_shots  shots_on_target  goals_per_shot  goals_per_sot  shot_on_targ

In [43]:
#Checking shape and data types
print(filtered_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1003 entries, 0 to 1002
Data columns (total 37 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   name                        1003 non-null   object 
 1   position                    1003 non-null   object 
 2   nation                      1000 non-null   object 
 3   age                         998 non-null    float64
 4   club                        1003 non-null   object 
 5   league                      1003 non-null   object 
 6   mins_played                 1003 non-null   int64  
 7   ninety_mins_played          1003 non-null   float64
 8   goals_scored                1003 non-null   int64  
 9   penalty_goals               1003 non-null   int64  
 10  non_pen_exp_goals           1003 non-null   float64
 11  exp_goals                   1003 non-null   float64
 12  total_shots                 1003 non-null   int64  
 13  shots_on_target             1003 

In [37]:
#Checking for duplicates
duplicate_names = filtered_df["name"][filtered_df["name"].duplicated(keep=False)]
print(duplicate_names)

4        Tammy Abraham
5        Tammy Abraham
6           Akor Adams
7           Akor Adams
27        Carles Aleñá
             ...      
961     Noah Weißhaupt
990     Nicolò Zaniolo
991     Nicolò Zaniolo
1001       Milan Đurić
1002       Milan Đurić
Name: name, Length: 100, dtype: object


In [39]:
#Grouping data to deal with players that have played for multiple clubs
aggregation_dict = {
    "position": lambda x: x.mode()[0] if not x.mode().empty else None,
    "nation": lambda x: x.mode()[0] if not x.mode().empty else None,
    "age": "mean",
    "club": lambda x: ", ".join(x.unique()),
    "league": lambda x: ", ".join(x.unique()),

    #Minutes & Matches
    "mins_played": "sum",
    "ninety_mins_played": "sum",

    #Goals & Assists
    "goals_scored": "sum",
    "penalty_goals": "sum",
    "non_pen_exp_goals": lambda x: np.average(x, weights=filtered_df.loc[x.index, "ninety_mins_played"]) if filtered_df.loc[x.index, "ninety_mins_played"].sum() != 0 else np.nan,
    "exp_goals": lambda x: np.average(x, weights=filtered_df.loc[x.index, "ninety_mins_played"]) if filtered_df.loc[x.index, "ninety_mins_played"].sum() != 0 else np.nan,
    "assists": "sum",
    "exp_assists": lambda x: np.average(x, weights=filtered_df.loc[x.index, "ninety_mins_played"]) if filtered_df.loc[x.index, "ninety_mins_played"].sum() != 0 else np.nan,

    #Shooting Metrics
    "total_shots": "sum",
    "shots_on_target": "sum",
    "goals_per_shot": lambda x: np.average(x, weights=filtered_df.loc[x.index, "total_shots"]) if filtered_df.loc[x.index, "total_shots"].sum() != 0 else np.nan,
    "goals_per_sot": lambda x: np.average(x, weights=filtered_df.loc[x.index, "shots_on_target"]) if filtered_df.loc[x.index, "shots_on_target"].sum() != 0 else np.nan,
    "shot_on_target_perc": lambda x: np.average(x, weights=filtered_df.loc[x.index, "total_shots"]) if filtered_df.loc[x.index, "total_shots"].sum() != 0 else np.nan,
    "avg_shot_distance": lambda x: np.average(x, weights=filtered_df.loc[x.index, "total_shots"]) if filtered_df.loc[x.index, "total_shots"].sum() != 0 else np.nan,

    #Creativity & Passing
    "key_passes": "sum",
    "shot_creation_per_90": lambda x: np.average(x, weights=filtered_df.loc[x.index, "ninety_mins_played"]) if filtered_df.loc[x.index, "ninety_mins_played"].sum() != 0 else np.nan,
    "goal_creation_per_90": lambda x: np.average(x, weights=filtered_df.loc[x.index, "ninety_mins_played"]) if filtered_df.loc[x.index, "ninety_mins_played"].sum() != 0 else np.nan,
    "pass_completion_perc": lambda x: np.average(x, weights=filtered_df.loc[x.index, "total_touches"]) if filtered_df.loc[x.index, "total_touches"].sum() != 0 else np.nan,

    #Progressive & Possession Play
    "progressive_carries": "sum",
    "total_carries": "sum",
    "total_touches": "sum",
    "touches_in_attacking_third": "sum",
    "touches_in_pen": "sum",
    "progressive_runs": "sum",
    "times_dispossessed": "sum",
    "passes_received": "sum",

    #Aerial Duels
    "aerial_duels_won": "sum",
    "aerial_duels_lost": "sum",
    "aerial_duel_win_perc": lambda x: np.average(x, weights=filtered_df.loc[x.index, "aerial_duels_won"] + filtered_df.loc[x.index, "aerial_duels_lost"]) if (filtered_df.loc[x.index, "aerial_duels_won"] + filtered_df.loc[x.index, "aerial_duels_lost"]).sum() != 0 else np.nan,

    #Defensive Actions
    "tackles": "sum",
    "tackles_and_interceptions": "sum"
}

#Applying groupby
grouped_df = filtered_df.groupby("name").agg(aggregation_dict).reset_index()

In [40]:
#Applying additional filter to include only strikers who have played more than 900 mins
striker_df = grouped_df[grouped_df["mins_played"] >= 900]
striker_df.shape

(493, 37)

In [42]:
#Checking for missing values
print(striker_df.isnull().sum())

name                           0
position                       0
nation                         0
age                            0
club                           0
league                         0
mins_played                    0
ninety_mins_played             0
goals_scored                   0
penalty_goals                  0
non_pen_exp_goals              0
exp_goals                      0
assists                        0
exp_assists                    0
total_shots                    0
shots_on_target                0
goals_per_shot                 3
goals_per_sot                 12
shot_on_target_perc            3
avg_shot_distance              3
key_passes                     0
shot_creation_per_90           0
goal_creation_per_90           0
pass_completion_perc           0
progressive_carries            0
total_carries                  0
total_touches                  0
touches_in_attacking_third     0
touches_in_pen                 0
progressive_runs               0
times_disp

We are missing values in the following columns:

- goals_per_shot
- goals_per_sot
- shot_on_target_perc
- avg_shot_distance
- aerial_duel_win_perc

In [44]:
#Saving the striker df to a csv for further eda and preprocessing
striker_df.to_csv("/Users/dan/PycharmProjects/Arsenal-Striker-Scouting/data/striker_df.csv")

EDA and preprocessing applied in notebook 02 eda